# Virtualization: virtual machines
_COSC 208, Introduction to Computer Systems, 2024-04-29_

## Announcements
* Attend VAP candidate talk tomorrow 11:30am-12:00pm
* Project 3 revisions due Friday @ 5pm
* Quiz 6 Wednesday – can start as early as 8:05am
    * 6.1: Describe the mechanisms operating systems use to allocate hardware resources and ensure errant processes do not take over the system
    * 6.2: Determine the possible outputs of C programs that involve multiple processes and/or threads
    * 6.3: Develop C programs that create and wait for processes and execute other programs
    * 6.4: Modify C programs to use threads
    * 6.5: Compare virtualization technologies
    * 5.1: Determine where code and data resides throughout a program’s lifecycle
    * 5.2: Determine how assembly and/or C code can be modified for more efficient use of storage
    * Optional:
        * 4.1: Interpret ARM assembly code
        * 4.2: Determine the relationship between C code and ARM assembly code

## Outline
* Warm-up
* Loop interchange
* Virtual machines
* Hypervisors

## Warm-up (Q3 from last class)

* Assume a cache can hold 3 entries and the following 15 data accesses occur: 
```
3, 4, 4, 5, 3, 2, 3, 4, 1, 4, 4, 2, 5, 2, 4
```
* Q1: _What is the sequence of hits, insertions, and replacements that occur when an **least recently used (LRU)** cache replacement algorithm is used?_

```
+3, +4, H4, +5, H3, -4/+2, H3, -5/+4, -2/+1, H4, H4, -3/+2, -1/+5, H2, H4
Hit ratio = 7/15 = 47%
```

<p style="height:10em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time take a few deep breaths to reduce stress.

<div style="page-break-after:always;"></div>

## Loop interchange

* Example

In [1]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int j = 0; j < 4; j++) {
        for (int k = 0; k < LEN; k += 4) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


* _Assume the values of all local variables are stored in registers (**not** the stack) and the value of `array` is `0x400`. What is the sequence of memory addresses that are accessed?_
    * First for loop: `0x400`, `0x404`, `0x408`, `0x40c`, `0x410`, `0x414`, `0x418`, `0x41c`, `0x420`, `0x424`, `0x428`, `0x42c`
    * Second for loop: `0x400`, `0x410`, `0x420`, `0x404`, `0x414`, `0x424`, `0x408`, `0x418`, `0x428`, `0x40c`, `0x41c`, `0x42c`, 
    * Notice that the first for loop accesses memory addresses in order, whereas the second for loop accesses addresses out of order
* _Now assume the system uses a cache that holds 2 entries which are each 16 bytes large. What is the sequence of hits and misses using a least recently used (LRU) replacement policy?_
    * First for loop: Miss (+0x4000), Hit, Hit, Hit, Miss (+0x4010), Hit, Hit, Hit, Miss (-0x4000/+0x4020), Hit, Hit, Hit
    * Second for loop: Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020), Miss (-0x4010/+0x4000), Miss (-0x4020/+0x4010), Miss (-0x4000/+0x4020)
    * Notice that the first for loop has three hits after each miss, whereas the second for loop is all misses
* _How could we modify the code to achieve a higher hit ratio?_ – loop interchange, i.e., swap inner and outer loops

In [2]:
#include <stdlib.h>
#include <stdio.h>
#define LEN 12
int main() {
    int *array = malloc(sizeof(int) * LEN);

    for (int i = 0; i < LEN; i++) {
        array[i] = i;
    }
    
    int sum = 0;
    for (int k = 0; k < LEN; k += 4) {
        for (int j = 0; j < 4; j++) {
            sum += array[j+k];
        }
    }
    printf("%d\n", sum);
}

66


* Q2: _Would loop interchange improve the efficiency of this code?_

In [ ]:
void hundreds() {
    int *nums = malloc(sizeof(int) * 1000);
    for (int i = 0; i < 1000; i+= 100) {
        for (int j = 0; j < 100; j++) {
            nums[i+j] = i;
        }
    }
}

    No

<p style="height:2em;"></p>

* Q3: _Would loop interchange improve the efficiency of this code?_

In [ ]:
void multiplication(int grid[][], int rows, int cols) {
    for (int c = 0; c < cols; c++) {
        for (int r = 0; r < rows; r++) {
            grid[r][c] = c * r;
        }
    }
}

    Yes

<div style="page-break-after:always;"></div>

## Virtual machines (VMs)

* Recall: _what is the purpose of an OS?_
    * Make computer hardware easy to use—e.g., an OS knows how to load an application's executable code from persistent storage (e.g., solid state drive (SSD)) into main memory, initialize the process's memory (code, heap, stack), and make the CPU execute the application's instructions
    * Support multiprocessing—i.e., running multiple applications simultaneously or concurrently
    * Allocate and manage hardware resources—e.g., decide when/which applications can use the CPU, decide when/which memory applications can use, prevent applications from stealing/accessing another application's CPU time or memory
* What is the purpose of a virtual machine?
    * Make computer hardware flexible to use
    * Support multiple OSes running concurrently
    * Allocate and manage hardware resources—e.g., decide when/which VMs can use the CPU, decide/when which memory VMs can use, prevent VMS from stealing/accessing another VM's CPU time or memory
* What is the motivation for using a virtual machine?
    * Run multiple OSes concurrently — _why would someone one to do this?_
        * Run software only supported by a specific OS
        * Software testing
        * Server consolidation
        * Security sandboxing
    * Flexibly use hardware resources

## Hypervisors

* _What additional software layer does a computer system need to support virtual machines?_ — hypervisor
* _Where does a hypervisor reside?_
    * Type 1: on top of hardware — e.g., KVM, Xen, Hyper-V, VMware vSphere
    * Type 2: on top of host OS — e.g., VirtualBox, VMware Workstation/Fusion
* _Why is a type 1 hypervisor better than a type 2 hypervisor?_
    * Resources can be oversubscribed
    * No overhead from host OS
* _Why is a type 2 hypervisor better than a type 1 hypervisor?_
    * Easy to use